In [ ]:

import numpy as np
from numba import jit, prange
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cProfile
import pstats

# --- Data Generation ---

# Parameters
num_qubits = 6
matrix_size = 2**num_qubits
num_repetitions = 1000
Mz_values = np.zeros((num_repetitions, matrix_size))


@jit(nopython=True, parallel=True)
eigenvalues = []
eigenvectors = []

for repetition in range(num_repetitions):


    #create random numbers for H.
    np.random.seed(repetition)
    J = np.random.uniform(low=-1, high=0, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                J[i, j] = 0
    J = (J + J.T)

    np.random.seed(repetition +10)
    K = np.random.uniform(low=-1, high=1, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                K[i, j] = 0
    K = (K + K.T)

    np.random.seed(repetition +20)
    h = np.random.uniform(low=-0.04, high=0.04)

    np.random.seed(repetition +30)
    g = np.random.uniform(low=-6, high=6, size=num_qubits)




    # create the matrix of H.
    matrix = np.zeros((matrix_size, matrix_size))
    for i in range(matrix_size):
        for j in range(matrix_size):
            H_value = 0
            for i1 in range(num_qubits):
                for j1 in range(num_qubits):
                    if j == i ^ (2 ** i1 + 2 ** j1):
#This part adds contributions to H_value based on the coupling constants K and the XOR operation (^).
#It checks if the bit-flipped indices i ^ (2 ** i1 + 2 ** j1) are equal to j and adds the corresponding K valeu.
                        H_value += K[i1, j1]
                    if j == i:
                        sign = 1
                        if (i & 2 ** i1) != 0:# & returns 1 if both the bits are 1, otherwise 0.
                            sign = -sign
                        if (i & 2 ** j1) != 0:
                            sign = -sign

                        H_value += sign * J[i1, j1]

            for i1 in range(num_qubits):
                if j == i ^ (2 ** i1):# external fields g for bit-flipped indices.
                    H_value += g[i1]

            if j == i:
                H_value += h # external field h when j is equal to i.
            matrix[i, j] = H_value




    # calculate eigenvalues and eigenvectors.
    eigenval, eigenvect = np.linalg.eigh(matrix)
   # print(eigenval)
   # print(eigenvect)
    min_eigenval = np.min(eigenval)
    min_eigenvec = eigenvect[np.argmin(eigenval), :]

    eigenvalues.append(min_eigenval)
    eigenvectors.append(min_eigenvec)

    #save the eigenvectors as a CSV file
eigenvectors_csv = np.array(eigenvectors)
np.savetxt('eigenvectors.csv', eigenvectors_csv, delimiter=',')
    #print
for repetition in range(num_repetitions):
    print("Repetition", repetition+1)
    print("Eigenvalue:", eigenvalues[repetition])
    print("Eigenvector:", eigenvectors[repetition])

    print()

  #creating the matrix of data, eigenvectors. each column is eigenvector of a certain H.
eigenvectors_matrix = np.column_stack(eigenvectors)

print("Eigenvectors Matrix:")
print(eigenvectors_matrix)


Mzt = []
for column in eigenvectors_matrix.T:
    i = 0
    Mz = 0
    for component in column:
        for n_prime in range(num_qubits):
            if (2 ** n_prime) & i != 0 :
                Mz += abs(component) ** 2  / num_qubits
          #Mz += abs(component) ** 2 * (2 * ((2 ** n_prime) & i) - 1) / num_qubits
        i += 1
    Mz=2*Mz-1
    Mzt.append(Mz)

Mzt_row = np.array(Mzt)
print("Mzt:", Mzt,"\n")
print(len(Mzt),"\n")
print(Mz)


import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt


# prepare data, i choose minmax as the book chose
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(eigenvectors_matrix)

# feature scaling, it calculate the number of columns and features
num_features = scaled_data.shape[1]

# Assuming you want to use the minimum number of samples between the two datasets
min_samples = min(len(scaled_data), len(Mzt))

# Adjust the size of both datasets
scaled_data = scaled_data[:min_samples]
Mzt = Mzt[:min_samples]

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_data, Mzt, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) #0.25*0.8=0.2

print("Training set size:", X_train.shape)
print("Validation set size:", X_val.shape)
print("Test set size:", X_test.shape)

#-----------------------------------
#part two: training
#-----------------------------------

import copy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
import itertools
import matplotlib.pyplot as plt
import time

class AdamOptimizer:
    def __init__(self, model, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.model = model
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0

    def compute_gradient(self, features, ansatz, num_qubits, labels):
        if self.m is None:
            self.m = np.zeros(len(ansatz.parameters))
            self.v = np.zeros(len(ansatz.parameters))

        gradient = np.zeros(len(ansatz.parameters))

        # Calculate gradients using finite differences
        weight_params = ansatz.parameters
        original_params = {param: np.random.uniform(0, 2*np.pi) for param in weight_params}
        for param_index, param in enumerate(weight_params):
            for i in [1, -1]:
                modified_params = original_params.copy()
                modified_params[param] += i * self.epsilon
                output_plus = self.model.forward(features, ansatz, num_qubits, modified_params)
                loss_plus = self.model.mse_loss(output_plus, labels)
                gradient[param_index] += i * (loss_plus - self.model.mse_loss(self.model.forward(features, ansatz, num_qubits, original_params), labels)) / (2 * self.epsilon)

        # Update biased first and second moments
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
        self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)

        # Bias correction
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        # Update parameters
        delta_params = -self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)

        for param_index, param in enumerate(weight_params):
            original_params[param] += delta_params[param_index]

        return original_params

class QuantumBrickWallModel:
    def __init__(self, num_qubits, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.num_qubits = num_qubits
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.circuit, self.parameters = self.build_qnn(num_qubits)
        self.optimizer = AdamOptimizer(self, learning_rate, beta1, beta2, epsilon)
        self.param_values = {param: np.random.uniform(0, 2*np.pi) for param in self.parameters}

    def build_qnn(self, num_qubits):
        qreg_q = QuantumRegister(num_qubits, 'q')
        ansatz = QuantumCircuit(qreg_q)
        ansatz.h(range(num_qubits))

        cnot_pairs = [(i, i+1) for i in range(0, num_qubits-1, 2)]
        cnot_pairs += [(i, i+1) for i in range(1, num_qubits-1, 2)]

        parameters = []
        layer = 3
        for i in range(layer):
            for j, (control, target) in enumerate(cnot_pairs):
                theta = Parameter(f'theta_{i}_{j}')
                phi = Parameter(f'phi_{i}_{j}')
                lam = Parameter(f'lam_{i}_{j}')
                gamma = Parameter(f'gamma_{i}_{j}')
                parameters += [theta, phi, lam, gamma]

                ansatz.cu(theta, phi, lam, gamma, qreg_q[control], qreg_q[target])

        return ansatz, parameters

    def forward(self, features, ansatz, num_qubits, param_bindings):
        output_list = []

        for state_vector in features:
            qc = QuantumCircuit(num_qubits)
            qc.initialize(state_vector, range(num_qubits))
            qc.compose(ansatz, inplace=True)
            cr = ClassicalRegister(1, name="c")
            qc.add_register(cr)
            qc.measure(num_qubits - 1, cr[0])

            bound_qc = qc.assign_parameters(param_bindings)

            backend = AerSimulator()
            job = backend.run(bound_qc, shots=1000)
            result = job.result()
            counts = result.get_counts(bound_qc)

            if '0' in counts and '1' in counts:
                exp_val = (counts['0'] - counts['1']) / sum(counts.values())
            elif '0' in counts:
                exp_val = counts['0'] / sum(counts.values())
            else:
                exp_val = -counts['1'] / sum(counts.values())

            output_list.append(exp_val)

        return output_list

    def mse_loss(self, output, labels):
        return np.mean(np.square(np.array(labels) - np.array(output)))


    def evaluate(self, features, labels):
        output = self.forward(features, self.circuit, self.num_qubits, self.param_values)
        loss = self.mse_loss(output, labels)
        return loss

    def predict(self, features):
        predictions = self.forward(features, self.circuit, self.num_qubits, self.param_values)
        return predictions

    def train(self, train_features, train_labels, val_features, val_labels, num_epochs):
        loss_history = []  # Initialize loss history
        val_loss_history = []  # Initialize validation loss history

        for epoch in range(num_epochs):
            # Training step
            output = self.forward(train_features, self.circuit, self.num_qubits, self.param_values)
            train_loss = self.mse_loss(output, train_labels)
            self.param_values = self.optimizer.compute_gradient(train_features, self.circuit, self.num_qubits, train_labels)

            # Validation step
            val_loss = self.evaluate(val_features, val_labels)

            # Append losses to history
            loss_history.append(train_loss)
            val_loss_history.append(val_loss)

            print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Validation Loss: {val_loss}")

        # Plot the training and validation loss
        plt.plot(loss_history, label='Train Loss')
        plt.plot(val_loss_history, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss Evolution')
        plt.legend()
        plt.show()

        # Return final validation loss for further evaluation
        return val_loss

    def test(self, test_features, test_labels):
        test_loss = self.evaluate(test_features, test_labels)
        print(f"Test Loss: {test_loss}")
        return


# ---------Run the code--------

# Create the model
model = QuantumBrickWallModel(num_qubits=num_qubits)

# Predictions on test set
predictions = model.predict(X_test)

# Record the start time
start_time = time.time()

# Train the model
val_loss = model.train(X_train, y_train, X_val, y_val, num_epochs=150)


# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Training completed in {elapsed_time:.2f} seconds")

# Test the model
test_loss = model.test(X_test, y_test)

